In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import pytz
import glob
from dateutil.parser import isoparse
from datetime import datetime, timedelta
from DirectoryGenerator import DirectoryGenerator

In [ ]:
dirGen = DirectoryGenerator()

In [ ]:
startDate = datetime.strptime(str(input("Start Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
startDate

In [ ]:
endDate = datetime.strptime(str(input("End Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
endDate

In [ ]:
totalDays = abs(endDate - startDate).days + 1
totalDays

In [ ]:
def parseTimestamp(timestamp):
    try:
        return isoparse(timestamp).astimezone(pytz.utc)
    except:
        return np.nan

In [ ]:
assignmentsDataCommentsFilesPath = str(input())

In [ ]:
assignmentsDataCommentsFiles = glob.glob(assignmentsDataCommentsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
assignmentsDataCommentsData = pd.DataFrame()
for assignmentsDataCommentsFile in assignmentsDataCommentsFiles:
    assignmentsDataCommentsData = assignmentsDataCommentsData.append(pd.read_csv(assignmentsDataCommentsFile), ignore_index = True)
    
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsData['metadata_event_time'] = assignmentsDataCommentsData['metadata_event_time'].apply(parseTimestamp)
assignmentsDataCommentsData.dropna(subset=['metadata_event_time'], inplace = True)
assignmentsDataCommentsData = assignmentsDataCommentsData.loc[(assignmentsDataCommentsData['metadata_event_time'] >= startDate) & (assignmentsDataCommentsData['metadata_event_time'] <= endDate)]
assignmentsDataCommentsData

In [ ]:
assignmentsDataSubmissionsFilesPath = str(input())

In [ ]:
assignmentsDataSubmissionsFiles = glob.glob(assignmentsDataSubmissionsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
assignmentsDataSubmissionsData = pd.DataFrame()
for assignmentsDataSubmissionsFile in assignmentsDataSubmissionsFiles:
    assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.append(pd.read_csv(assignmentsDataSubmissionsFile), ignore_index = True)
    
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData['metadata_event_time'] = assignmentsDataSubmissionsData['metadata_event_time'].apply(parseTimestamp)
assignmentsDataSubmissionsData.dropna(subset=['metadata_event_time'], inplace = True)
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.loc[(assignmentsDataSubmissionsData['metadata_event_time'] >= startDate) & (assignmentsDataSubmissionsData['metadata_event_time'] <= endDate)]
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData.drop(['body_submitted_at', 'body_attempt'], axis = 1, inplace = True)
assignmentsDataSubmissionsData

In [ ]:
assignmentInfoData = pd.read_csv(str(input()))
assignmentInfoData

In [ ]:
#assignmentInfoData.drop(['metadata_event_time', 'body_context_id', 'body_due_at'], axis = 1, inplace = True)
#assignmentInfoData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.merge(assignmentInfoData, on = ['metadata_context_id', 'body_assignment_id'])
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataCommentsData = assignmentsDataCommentsData.merge(assignmentsDataSubmissionsData[['body_submission_id', 'body_assignment_id', 'metadata_context_id', 'body_user_id']].rename(columns={'body_user_id': 'metadata_user_id'}), on=['body_submission_id', 'metadata_context_id', 'metadata_user_id'])
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsData.drop(['body_submission_id', 'submission_comments_count'], axis = 1, inplace = True)
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsData['metadata_event_time'] = assignmentsDataCommentsData['metadata_event_time'].apply(str)
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsData.groupby(['metadata_context_id', 'metadata_user_id']).agg({"metadata_event_time": list}).reset_index().to_csv("AssignmentComments_Intercession_2022.csv", header = True, index = False)